In [23]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [24]:
import json
import glob2
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import regex as re
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import optim
import os
import pickle
import random
!pip install transformers
from transformers import BertTokenizer   
from transformers import BertModel

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
# class Node():
#   def __init__(self,x=None,y=None,id=None,node1=None,node2=None):
#     global gb
#     if(id is None and (node1 is None or node2 is None)):
#       raise Exception("To create non virtual nodes, id is required")
#     if(id is not None):
#       self.id=str(id)
#       self.is_leaf_node=True
#     else:
#       gb.varint+=1
#       self.id='vir_'+ str(gb.varint)
#       self.node1=node1
#       self.node2=node2  
#       if('vir_' in node1.id and 'vir_' in node2.id):
#         self.is_stance_node=False
#       else:
#         self.is_stance_node=True
#       self.is_leaf_node=False  
#     self.x=x
#     self.y=y
#     self.c=None
#     self.h=None 
class Node():
  def __init__(self,x=None,y=None,id=None,children=None):
     self.x=x
     self.id=id
     self.y=y
     self.children=children
     self.h=None
     self.c=None    

In [26]:
class Global:
    def __init__(self):
      # self.varint=0
      # self.g_parent=None
      self.g_nodes=[] 
    def reset(self):
      # self.varint=0
      # self.g_parent=None
      self.g_nodes=[]

def get_key_value(dicts,search_id,keys=None):
  res={}
  for d in dicts:
    if('id' in d.keys()):
      if(str(d['id'])==str(search_id)):
        for k in keys:
          res[k]=d[k]
        break
    else:
      if(search_id in d.keys()):  
        return d[search_id]
      else:
        return None        
  return list(res.values()) 

def uniform_embed(size):
  return np.random.uniform(low=-0.5, high=0.5, size=(size))

def create_tree(tree,tweets=None,stance_dict=None,parent=None,embed_size=2400):
  global gb
  children=[]
  for key,value in tree.items():
    if(type(value)==dict):      
      child=create_tree(value,tweets,stance_dict,parent=key,embed_size=embed_size)
      embed=get_key_value(dicts=tweets,search_id=key,keys=['text'])
      if(len(embed)>0 and len(embed[0])>0):
        embed=embed[0]
      else:
        embed=uniform_embed(embed_size)       
      parent_node=Node(id=key,x=embed,children=child)
      children.append(parent_node)
      gb.g_nodes.append(parent_node)
    else:
      embed=get_key_value(dicts=tweets,search_id=key,keys=['text'])
      if(len(embed)>0 and len(embed[0])>0):
        embed=embed[0]
      else:
        embed=uniform_embed(embed_size)            
      child=Node(id=key,x=embed)
      children.append(child)
      gb.g_nodes.append(child) 
  return children    
  #   children.append(child) 
  # if(gb.g_parent is not None):
  #   arr=[gb.g_parent]
  #   arr.extend(children)
  #   children=arr 
  # if(parent is not None):
  #   nodes=[]
  #   for child in children:
  #     if('vir_' not in str(parent) and 'vir_' in child.id):
  #       nodes.append(child)
  #       continue
  #     embed=get_key_value(dicts=tweets,search_id=parent,keys=['text']) 
  #     if(len(embed)>0 and len(embed[0])>0):
  #       embed=embed[0]  
  #     else:
  #       embed=uniform_embed(embed_size)      
  #     parent_node=Node(id=parent,x=embed) 
  #     stance=None
  #     if('var_' not in parent_node.id and 'var_' not in child.id):
  #       stance=get_key_value(dicts=[stance_dict],search_id=child.id) 
  #     node=Node(node1=parent_node,node2=child,y=stance)
  #     # print(parent,child.id,node.id)
  #     gb.g_nodes.extend([parent_node,child,node])
  #     nodes.append(node)
  #   node1=nodes[0] 
  #   for node_iter in range(1,len(nodes)):
  #     node2=nodes[node_iter]
  #     node=Node(node1=node1,node2=node2)
  #     # print(node1.id,node2.id,node.id)
  #     gb.g_nodes.extend([node])
  #     node1=node 
  #   embed=get_key_value(dicts=tweets,search_id=parent,keys=['text']) 
  #   if(len(embed)>0 and len(embed[0])>0):
  #     embed=embed[0]  
  #   else:
  #     embed=uniform_embed(embed_size)            
  #   gb.g_parent=Node(id=parent,x=embed)
  #   return node1
  # return children     

# SKP Encoder python2

In [ ]:
# !git clone https://github.com/ryankiros/skip-thoughts.git
!wget http://www.cs.toronto.edu/~rkiros/models/dictionary.txt -P skip-thoughts
!wget http://www.cs.toronto.edu/~rkiros/models/utable.npy -P skip-thoughts
!wget http://www.cs.toronto.edu/~rkiros/models/btable.npy -P skip-thoughts
!wget http://www.cs.toronto.edu/~rkiros/models/uni_skip.npz -P skip-thoughts
!wget http://www.cs.toronto.edu/~rkiros/models/uni_skip.npz.pkl -P skip-thoughts
!wget http://www.cs.toronto.edu/~rkiros/models/bi_skip.npz -P skip-thoughts
!wget http://www.cs.toronto.edu/~rkiros/models/bi_skip.npz.pkl -P skip-thoughts

--2020-06-21 03:49:17--  http://www.cs.toronto.edu/~rkiros/models/dictionary.txt
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7996547 (7.6M) [text/plain]
Saving to: ‘skip-thoughts/dictionary.txt’

dictionary.txt      100%[===================>]   7.63M  11.0MB/s    in 0.7s    

2020-06-21 03:49:17 (11.0 MB/s) - ‘skip-thoughts/dictionary.txt’ saved [7996547/7996547]

--2020-06-21 03:49:19--  http://www.cs.toronto.edu/~rkiros/models/utable.npy
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2342138474 (2.2G)
Saving to: ‘skip-thoughts/utable.npy’

utable.npy          100%[===================>]   2.18G  84.4MB/s    in 28s     

2020-06-21 03:49:47 (79.0 MB/s) - ‘skip-thoughts/u

In [ ]:
# %cd skip-thoughts
# import importlib
# importlib.reload(skipthoughts)
import skipthoughts
model = skipthoughts.load_model()
encoder = skipthoughts.Encoder(model)

[Errno 2] No such file or directory: 'skip-thoughts'
/content/skip-thoughts


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


# SKP encoder python3

In [ ]:
!wget "http://download.tensorflow.org/models/skip_thoughts_bi_2017_02_16.tar.gz"
!tar -xvf skip_thoughts_bi_2017_02_16.tar.gz
!rm skip_thoughts_bi_2017_02_16.tar.gz

--2020-06-21 06:02:01--  http://download.tensorflow.org/models/skip_thoughts_bi_2017_02_16.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.214.128, 2607:f8b0:400c:c0b::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.214.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5680270817 (5.3G) [application/x-gzip]
Saving to: ‘skip_thoughts_bi_2017_02_16.tar.gz’

skip_thoughts_bi_20 100%[===================>]   5.29G  58.6MB/s    in 74s     

2020-06-21 06:03:15 (72.9 MB/s) - ‘skip_thoughts_bi_2017_02_16.tar.gz’ saved [5680270817/5680270817]

skip_thoughts_bi_2017_02_16/
skip_thoughts_bi_2017_02_16/model.ckpt-500008.data-00000-of-00001
skip_thoughts_bi_2017_02_16/vocab.txt
skip_thoughts_bi_2017_02_16/model.ckpt-500008.index
skip_thoughts_bi_2017_02_16/embeddings.npy
skip_thoughts_bi_2017_02_16/model.ckpt-500008.meta


In [ ]:
!git clone https://github.com/elvisyjlin/skip-thoughts.git
%tensorflow_version 1.x
%cd skip-thoughts/

Cloning into 'skip-thoughts'...
remote: Enumerating objects: 35, done.
remote: Total 35 (delta 0), reused 0 (delta 0), pack-reused 35
Unpacking objects: 100% (35/35), done.
TensorFlow 1.x selected.
/content/skip-thoughts


In [ ]:
from skip_thoughts import configuration
from skip_thoughts import encoder_manager

VOCAB_FILE = "skip_thoughts_bi_2017_02_16/vocab.txt"
EMBEDDING_MATRIX_FILE = "skip_thoughts_bi_2017_02_16/embeddings.npy"
CHECKPOINT_PATH = "skip_thoughts_bi_2017_02_16/model.ckpt-500008"

encoder = encoder_manager.EncoderManager()
encoder.load_model(configuration.model_config(bidirectional_encoder=True),
                   vocabulary_file=VOCAB_FILE,
                   embedding_matrix_file=EMBEDDING_MATRIX_FILE,
                   checkpoint_path=CHECKPOINT_PATH)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


INFO:tensorflow:Reading vocabulary from skip_thoughts_bi_2017_02_16/vocab.txt

INFO:tensorflow:Loaded vocabulary with 930914 words.
INFO:tensorflow:Loading embedding matrix from skip_thoughts_bi_2017_02_16/embeddings.npy
INFO:tensorflow:Loaded embedding matrix with shape (930914, 620)
INFO:tensorflow:Building model.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.


Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updat

# DeepMoji Encoder

In [ ]:
# for deepMoji
!git clone https://github.com/zzsza/DeepMoji-Python3.git
%cd DeepMoji-Python3/DeepMoji-master
!python scripts/download_weights.py
!pip install emoji
# replace downloaded encode_texts.py file with modified file

Cloning into 'DeepMoji-Python3'...
remote: Enumerating objects: 85, done.
remote: Total 85 (delta 0), reused 0 (delta 0), pack-reused 85
Unpacking objects: 100% (85/85), done.
/content/DeepMoji-Python3/DeepMoji-master
About to download the pretrained weights file from https://www.dropbox.com/s/xqarafsl6a8f9ny/deepmoji_weights.hdf5?dl=0#
The size of the file is roughly 85MB. Continue? [y/n]
y
Downloading...
Running system call: wget https://www.dropbox.com/s/xqarafsl6a8f9ny/deepmoji_weights.hdf5?dl=0# -O /content/DeepMoji-Python3/DeepMoji-master/model/deepmoji_weights.hdf5
--2020-06-22 07:25:53--  https://www.dropbox.com/s/xqarafsl6a8f9ny/deepmoji_weights.hdf5?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.1, 2620:100:6023:1::a27d:4301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xqarafsl6a8f9ny/deepmoji_weights.hdf5 [following]
--2020-06-22 07:25:53--  https

# BERT Encoder

In [27]:
class SentenceClassifier(nn.Module):
  def __init__(self):
      super(SentenceClassifier, self).__init__()
      self.bert_layer = BertModel.from_pretrained('/content/drive/My Drive/BERT_CONFIG')
      self.linear = nn.Linear(768*2, 3)
      self.softmax = nn.Softmax()

  def forward(self,seq,mask):
      _,cls = self.bert_layer(seq, attention_mask = mask)
      return cls

# Encoders

In [28]:
# import examples.encode_texts

class GLOVE:
  def __init__(self,embedding_file):
    self.vocab={}
    self.unk=[0 for i in range(300)]
    with open(embedding_file, 'rt', encoding='utf-8') as f:
      for line in f:
        splitline=line.split()
        self.vocab[splitline[0]]=[float(value) for value in splitline[1:]]
    self.size=300    
  def embed(self,tokens):
    embed = [self.vocab.get(token,self.unk) for token in tokens]
    embed = [sum(x)/len(x) for x in zip(*embed)]
    return embed

class DeepMoji:
  def __init__(self):
    self.size=2304
  def embed(self,tokens):
    embed=examples.encode_texts.encode(np.array([''.join(tokens)]))
    return embed.reshape(2304)

class SKP:
  def __init__(self,encoder):
    self.encoder=encoder
    self.size=2400
  def embed(self,tokens):
    embed=self.encoder.encode(tokens) 
    embed=[sum(x)/len(x) for x in zip(*embed)]
    return embed

class skpemt:
  def __init__(self,encoder):
    self.encoder=encoder
    self.size=2400+2304
  def embed(self,tokens):
    encoding=self.encoder.encode(tokens) 
    encoding=[sum(x)/len(x) for x in zip(*encoding)]
    deepmoji_encoding=examples.encode_texts.encode(np.array([''.join(tokens)]))
    deepmoji_encoding= list(deepmoji_encoding.reshape(2304))
    encoding.extend(deepmoji_encoding)
    return encoding 

class BERT:
  def __init__(self):
    self.model=SentenceClassifier()
    self.model.load_state_dict(torch.load('/content/drive/My Drive/BERT_CONFIG/multinli_model.pt'))
    self.size=768
    self.tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/BERT_CONFIG')    
  def tokenize(self,tokens):
    return [self.tokenizer.convert_tokens_to_ids('[CLS]')]\
      +[self.tokenizer.convert_tokens_to_ids(tok) for tok in tokens]
  def embed(self,tokens):
    tokenized=torch.tensor(self.tokenize(tokens)).view(1,-1)
    mask=torch.tensor([1 for i in range(tokenized.size(-1))]).view(1,-1)
    return self.model(tokenized,mask)            

# Preprocessing Input

In [29]:
def create_datapoint(source,reply_list,struct,stance_dict,embedding):
    tweets=reply_list+[source]
    create_tree(struct,tweets,stance_dict,embed_size=embedding.size)

def preprocess(tweet):
    # remove @mentions, RT,MT,DM,PRT,HT,CC, URLs
    contractions = { 
      "n't": "not",
      "'ve": "have",
      "'d": "would",
      "'ll": "will",
      "'m": "am",
      "ma'am": "madam",
      "'re": "they are"
      }
    text=tweet.lower()  
    tokens=text.split()
    # print(tokens)
    for indx,token in enumerate(tokens):
        for contra in contractions.keys():
            if(contra in token):
                tokens[indx]=' '.join([token.replace(contra,''),contractions[contra]])
    text=' '.join(tokens)
    # Format words and remove unwanted characters    
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'([@?])(\w+)\b', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text=text.replace('mt ',' ').replace('rt ',' ').replace('dm ',' ').replace('prt ',' ').replace('ht ',' ').replace('cc ',' ')
    tokens = text.split()
    stops = set(stopwords.words("english"))
    tokens = [w for w in tokens if not w in stops]
    return tokens

In [ ]:
text='MT @euronews France: 10 dead after shooting at HQ of satirical weekly #CharlieHebdo. If Zionists/Jews did this they\'d be nuking Israel'
preprocess(text)

['france',
 '10',
 'dead',
 'shooting',
 'hq',
 'satirical',
 'weekly',
 'charliehebdo',
 'zionists',
 'jews',
 'would',
 'nuking',
 'israel']

# Model

In [89]:
class TreeLSTMCell(nn.Module):
  def __init__(self,x_size,h_size):
    super(TreeLSTMCell, self).__init__()
    self.W_iou = nn.Linear(x_size, 3 * h_size, bias=False)
    self.U_iou = nn.Linear(h_size, 3 * h_size, bias=False)
    self.b_iou = nn.Parameter(torch.zeros(1, 3 * h_size))
    self.U_f = nn.Linear(h_size, h_size)
    self.b_f = nn.Parameter(torch.zeros(1, h_size))      
    self.h=torch.zeros(1,h_size)
    self.c=torch.zeros(1,h_size)
    self.sigmoid=nn.Sigmoid()
    self.tanh=nn.Tanh()
    self.conv=nn.Conv1d(in_channels=1,out_channels=1, kernel_size=2, stride=1,padding=1)
    self.maxpool=nn.AdaptiveMaxPool1d(768)
    self.stance_linear=nn.Linear(h_size,4)
    self.veracity_linear=nn.Linear(h_size,3)
    self.softmax=nn.Softmax()
    self.h_size=h_size
  def reset_h_c(self):
    self.h=torch.zeros(1,self.h_size)
    self.c=torch.zeros(1,self.h_size)    
  def forward(self,node,is_last=False):      
    if(node.children is None):
      x=torch.tensor(node.x,dtype=torch.float)
      x_iou=self.W_iou(x) 
      u_iou=self.U_iou(self.h)
      b_iou=self.b_iou      
      xs=torch.chunk(x_iou,3,-1)
      us=torch.chunk(u_iou,3,-1)
      bs=torch.chunk(b_iou,3,-1)       
      i=self.sigmoid(xs[0]+us[0]+bs[0])
      o=self.sigmoid(xs[1]+us[1]+bs[1])
      u=self.sigmoid(xs[2]+us[2]+bs[2])
      self.c=i*u
      self.h=o*self.tanh(self.c)
      node.h=self.h
      node.c=self.c
    else:
      h_stack=torch.stack([child.h for child in node.children])
      u_f=self.U_f(h_stack.view(-1,self.h_size))
      f=self.sigmoid(u_f+self.b_f)
      # print(h_stack.shape)
      h_hat,_=torch.max(self.maxpool(self.conv(h_stack)),dim=0,keepdim=True)
      u_iou=self.U_iou(h_hat.view(1,-1))

      us=torch.chunk(u_iou,3,-1)
      bs=torch.chunk(self.b_iou,3,-1)            
      i=self.sigmoid(us[0]+bs[0])
      o=self.sigmoid(us[1]+bs[1])
      u=self.sigmoid(us[2]+bs[2])
      self.c=i*u + torch.sum(f.view(-1,self.h_size)*torch.stack([child.c for child in node.children]).view(-1,self.h_size),0)
      self.h=o*self.tanh(self.c)
      node.h=self.h
      node.c=self.c
      if(is_last):
        veracity=self.softmax(self.veracity_linear(self.h))
        print('veracity:{}'.format(veracity))
        return veracity 
      stance=self.softmax(self.stance_linear(self.h))
      return stance          

# Loading Data

In [30]:
veracity_enum = { 
  'true':0,
  'false':1,
  'unverified':2
}

stance_enum={
    'support':0,
    'comment':1,
    'query':2,
    'deny':3
}

In [91]:
class Event:
  def __init__(self,path):
    self.path=path
    self.treelist=[]
  def create_tree(self,gb):
    dirs=os.listdir(self.path)
    for d in dirs:
      if(d=='.DS_Store'):
        continue
      source=glob2.glob(self.path+'/'+ d + '/source-tweet/*.json')[0]
      replies=glob2.glob(self.path+'/'+ d + '/replies/*.json')
      structure=self.path+'/'+ d + '/structure.json'
      gb.reset()
      reply_list=[]
      with open(source,'r') as f:
        source=json.load(f,parse_int=None)
      processed_text= preprocess(source['text'])  
      if(len(processed_text)>0):  
        source={'id':source['id'],'text':embedding.embed(processed_text)}  
      else:
        source={'id':source['id'],'text':processed_text}        
      for reply in replies:
        with open(reply,'r') as f:
          reply=json.load(f,parse_int=None)
        processed_text= preprocess(reply['text'])
        if(len(processed_text)>0):
          reply={'id':reply['id'],'text':embedding.embed(processed_text)}
        else:
          reply={'id':reply['id'],'text':processed_text} 
        reply_list.append(reply)
        # print(reply_list)      
      with open(structure,'r') as f:
        struct=json.load(f,parse_int=None)
      create_datapoint(source,reply_list,struct,stance_dict,embedding=embedding)
      self.treelist.append((source['id'],gb.g_nodes))      

In [92]:
# load stance labels
rumour_files=glob2.glob('/content/drive/My Drive/rumoureval-data/traindev/*subtaskA*.json')
stance_dict={}
for file in rumour_files:
  with open(file,'r') as f:
    stance_dict.update(json.load(f,parse_int=None))
for k,v in stance_dict.items():
  stance_dict[k]=stance_enum[v]

#load veracity labels
rumour_files=glob2.glob('/content/drive/My Drive/rumoureval-data/traindev/*subtaskB*.json')
veracity_dict={}
for file in rumour_files:
  with open(file,'r') as f:
    veracity_dict.update(json.load(f,parse_int=None))
for k,v in veracity_dict.items():
  veracity_dict[k]=veracity_enum[v]

In [ ]:
embedding=BERT()
with open('/content/drive/My Drive/rumoureval-data/charliehebdo/553184482241814530/replies/553184702530453505.json','r') as f:
  source=json.load(f,parse_int=None)
source={'id':source['id'],'text':embedding.embed(preprocess(source['text']))} 
source
# type(encoder.encode(['maybe', 'they', 'want', 'to', 'get' ,'caught']))

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


{'id': 553184702530453505,
 'text': tensor([[-0.3701, -0.2217,  0.9532,  0.5189,  0.8301,  0.2629,  0.8415,  0.6777,
           0.9937, -0.9796,  0.9742, -0.8071,  0.9026, -0.2117,  0.9733, -0.5808,
           0.0143, -0.3522,  0.3882, -0.8300,  0.7825,  0.0173,  0.8494,  0.6325,
           0.3077, -0.8371, -0.4423,  0.9895,  0.9898,  0.4993, -0.1705, -0.5002,
          -0.8806,  0.4054,  0.8998, -0.1324, -0.6142, -0.3484, -0.4314,  0.5374,
          -0.9941,  0.6816,  0.7429, -0.9557,  0.6775,  0.0691, -0.0355, -0.3049,
          -0.6994, -0.9876, -0.4933, -0.8455, -0.5568,  0.0603, -0.3631, -0.1101,
          -0.6004, -0.7368, -0.5735,  0.1829,  0.0956, -0.5329,  0.7888, -0.8245,
          -0.9179, -0.9017, -0.1838, -0.5882, -0.3484, -0.6698,  0.4890, -0.0192,
           0.2107, -0.9385, -0.8362, -0.6202, -0.2919,  0.2919,  0.1369, -0.9912,
          -0.9393, -0.8425, -0.2211,  0.8951, -0.6955,  0.0378, -0.8312,  0.2405,
          -0.9967,  0.0448,  0.6354,  0.3575, -0.5427,  0.5747,

In [93]:
import pickle
# import importlib
# importlib.reload(examples.encode_texts)

def dump_embedding(typ):
  root_path='/content/drive/My Drive/rumoureval-data/'
  for e in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
    # event_list=[]
    ev=Event(root_path+e)
    ev.create_tree(gb)
    with open('/content/drive/My Drive/Tree_LSTM/data_{}_{}.pkl'.format(typ,e),'wb') as f:
      pickle.dump(ev,f)

# embedding=SKP(encoder)
# embedding=DeepMoji()
embedding=BERT()
gb=Global()
# dump_embedding('skp')
dump_embedding('bert')      

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [39]:
# with open('/content/drive/My Drive/data.pkl','rb') as f:
#   event_list=pickle.load(f)
for node in gb.g_nodes:
  if(node.children is not None):
    for child in node.children:
      print('parent:{}; child{}'.format(node.id,child.id))
  print(node.id)    
# [node.id for node in gb.g_nodes]

553184702530453505
553184777315287040
553184841873633280
553184912258654208
553185239791857664
553185273639895040
553185403650322432
553185449150541824
553186074311528448
553187655434108928
553199414659907584
parent:553199295588212736; child553199414659907584
553199295588212736
parent:553197339813220352; child553199295588212736
553197339813220352
parent:553187777689694208; child553197339813220352
553187777689694208
553187813668425728
553198944998543360
parent:553197749240221699; child553198944998543360
553197749240221699
553549691129192448
parent:553549483984711681; child553549691129192448
553549483984711681
parent:553498348674875393; child553549483984711681
553498348674875393
parent:553188394113986561; child553197749240221699
parent:553188394113986561; child553498348674875393
553188394113986561
553188436178264066
553200322231808001
parent:553189048052105216; child553200322231808001
553189048052105216
553192939888058369
553195335661912064
553195879692517376
553196247319064576
parent:55

In [ ]:
# Balancing each class for rumour labels
def balance_embedding(typ):
  ev_label_count=[]
  for e in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
    with open('/content/drive/My Drive/DeepMoji/data_{}_{}.pkl'.format(typ,e),'rb') as f:
      ev=pickle.load(f)
    label={0:0,1:0,2:0}   
    # for event in ev:
    for tree in ev.treelist:
      label[veracity_dict[str(tree[0])]]+=1
    ev_label_count.append((e,label))    
  print(ev_label_count)
  for e in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
    with open('/content/drive/My Drive/DeepMoji/data_{}_{}.pkl'.format(typ,e),'rb') as f:
      ev=pickle.load(f) 
    # for event in ev:
    lbl_0_list=[]
    lbl_1_list=[]
    lbl_2_list=[]
    for tree in ev.treelist: 
      if(veracity_dict[str(tree[0])]==0):
        lbl_0_list.append(tree)      
      elif(veracity_dict[str(tree[0])]==1):
        lbl_1_list.append(tree)
      elif(veracity_dict[str(tree[0])]==2):
        lbl_2_list.append(tree)
    if(e=='charliehebdo'):
      for i in range(27):
        ev.treelist.append(random.sample(lbl_1_list,1)[0])
      for i in range(16):
        ev.treelist.append(random.sample(lbl_2_list,1)[0])
    elif(e=='sydneysiege'):
      for i in range(38):
        ev.treelist.append(random.sample(lbl_1_list,1)[0])
      for i in range(44):
        ev.treelist.append(random.sample(lbl_2_list,1)[0])   
    elif(e=='germanwings-crash'):
      for i in range(9):
        ev.treelist.append(random.sample(lbl_2_list,1)[0]) 
    elif(e=='ferguson'):
      for i in range(42):
        ev.treelist.append(random.sample(lbl_0_list,1)[0])
    elif(e=='ottawashooting'):
      for i in range(40):
        ev.treelist.append(random.sample(lbl_1_list,1)[0])    
      for i in range(23):
        ev.treelist.append(random.sample(lbl_2_list,1)[0])    
    with open('/content/drive/My Drive/DeepMoji/data_{}_{}_balanced.pkl'.format(typ,e),'wb') as f:
      pickle.dump(ev,f) 

  ev_label_count=[]
  for e in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
    with open('/content/drive/My Drive/DeepMoji/data_{}_{}_balanced.pkl'.format(typ,e),'rb') as f:
      ev=pickle.load(f)
    label={0:0,1:0,2:0}   
    # for event in ev:
    for tree in ev.treelist:
      label[veracity_dict[str(tree[0])]]+=1
    ev_label_count.append((e,label))    
  print(ev_label_count) 

# balance_embedding('skp')                                                                   
# balance_embedding('deepmoji')
balance_embedding('bert')                                                                   

[('charliehebdo', {0: 39, 1: 12, 2: 23}), ('sydneysiege', {0: 51, 1: 13, 2: 7}), ('germanwings-crash', {0: 10, 1: 12, 2: 3}), ('ferguson', {0: 2, 1: 0, 2: 44}), ('ottawashooting', {0: 35, 1: 10, 2: 13})]


/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


[('charliehebdo', {0: 39, 1: 39, 2: 39}), ('sydneysiege', {0: 51, 1: 51, 2: 51}), ('germanwings-crash', {0: 10, 1: 12, 2: 12}), ('ferguson', {0: 44, 1: 0, 2: 44}), ('ottawashooting', {0: 35, 1: 50, 2: 36})]


In [43]:
event_labels=['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']
# change type for embedding skp,deepmoji,bert,mix(deepmoji+skp)
def train_loader(val_index):
  for e in [l for i,l in enumerate(event_labels) if i!=val_index]:
    with open('/content/drive/My Drive/Tree_LSTM/data_bert_{}.pkl'.format(e),'rb') as f:
      ev=pickle.load(f)
    yield ev,e

def val_loader(val_index):
  for e in [l for i,l in enumerate(event_labels) if i==val_index]:
    with open('/content/drive/My Drive/Tree_LSTM/data_bert_{}.pkl'.format(e),'rb') as f:
      ev=pickle.load(f)
    yield ev,e

# Training

In [94]:
# skp:
# model=TreeLSTMCell(2400,2400)
# deepmoji
model=TreeLSTMCell(768,768)

optimizer = optim.Adam(model.parameters(), lr=.008)
criterion = nn.CrossEntropyLoss()

for e in range(30):
  val_index=e%5
  train_loss=0
  for event,lbl in train_loader(val_index):
    print('training on {}...'.format(lbl))
    for tree in event.treelist:
        loss=0
        optimizer.zero_grad()
        nodes=tree[1]
        for i in range(len(nodes)):
          nodes[i].h=None
          nodes[i].c=None
        model.reset_h_c()  
        for i in range(len(nodes)):
          if(i!=(len(nodes)-1)):
            result=model(nodes[i])
            if(result is not None and nodes[i].y is not None):
              loss+=criterion(result,torch.tensor(nodes[i].y).view(-1))
          else:
            result=model(nodes[i],is_last=True)  
            print(result,veracity_dict[str(tree[0])],node.id,node.children)        
            loss+=criterion(result,torch.tensor(veracity_dict[str(tree[0])],dtype=torch.long).view(-1))            
        train_loss+=loss.data.item()        
        loss.backward()
        optimizer.step()
  print('train loss : {}'.format(train_loss))     
  with torch.no_grad():
    loss=0
    for event,lbl in val_loader(val_index):
      print('testing on {}...'.format(lbl))
      for tree in event.treelist:
        nodes=tree[1]
        for i in range(len(nodes)):
          nodes[i].h=None
          nodes[i].c=None
        model.reset_h_c()  
        for i in range(len(nodes)):
          if(i!=(len(nodes)-1)):
            result=model(nodes[i])
            if(result is not None and nodes[i].y is not None):
              loss+=criterion(result,torch.tensor(nodes[i].y).view(-1)).data.item()
          else:
            result=model(nodes[i],is_last=True)
            loss+=criterion(result,torch.tensor(veracity_dict[str(tree[0])],dtype=torch.long).view(-1)).data.item() 
    print('test loss : {}'.format(loss))
    torch.save(model.state_dict(), '/content/drive/My Drive/model_bert.pt')

training on sydneysiege...
veracity:tensor([[0.2907, 0.3749, 0.3344]], grad_fn=<SoftmaxBackward>)
tensor([[0.2907, 0.3749, 0.3344]], grad_fn=<SoftmaxBackward>) 0 553184482241814530 [<__main__.Node object at 0x7f32c645d048>, <__main__.Node object at 0x7f32c70cf5c0>, <__main__.Node object at 0x7f32c70cf940>, <__main__.Node object at 0x7f32c70cf7b8>, <__main__.Node object at 0x7f32c70cf5f8>, <__main__.Node object at 0x7f32c70cf6d8>, <__main__.Node object at 0x7f32c70cf898>, <__main__.Node object at 0x7f32c70cf668>, <__main__.Node object at 0x7f32c70cf978>, <__main__.Node object at 0x7f32c70cf908>, <__main__.Node object at 0x7f32c70cf588>, <__main__.Node object at 0x7f32c70cf828>, <__main__.Node object at 0x7f32ca43e7f0>, <__main__.Node object at 0x7f32ca43edd8>, <__main__.Node object at 0x7f32ca43efd0>, <__main__.Node object at 0x7f32ca43eb70>, <__main__.Node object at 0x7f32ca43e358>, <__main__.Node object at 0x7f32ca43ecf8>, <__main__.Node object at 0x7f32ca43e320>]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


veracity:tensor([[9.9999e-01, 3.1789e-06, 3.4650e-06]], grad_fn=<SoftmaxBackward>)
tensor([[9.9999e-01, 3.1789e-06, 3.4650e-06]], grad_fn=<SoftmaxBackward>) 0 553184482241814530 [<__main__.Node object at 0x7f32c645d048>, <__main__.Node object at 0x7f32c70cf5c0>, <__main__.Node object at 0x7f32c70cf940>, <__main__.Node object at 0x7f32c70cf7b8>, <__main__.Node object at 0x7f32c70cf5f8>, <__main__.Node object at 0x7f32c70cf6d8>, <__main__.Node object at 0x7f32c70cf898>, <__main__.Node object at 0x7f32c70cf668>, <__main__.Node object at 0x7f32c70cf978>, <__main__.Node object at 0x7f32c70cf908>, <__main__.Node object at 0x7f32c70cf588>, <__main__.Node object at 0x7f32c70cf828>, <__main__.Node object at 0x7f32ca43e7f0>, <__main__.Node object at 0x7f32ca43edd8>, <__main__.Node object at 0x7f32ca43efd0>, <__main__.Node object at 0x7f32ca43eb70>, <__main__.Node object at 0x7f32ca43e358>, <__main__.Node object at 0x7f32ca43ecf8>, <__main__.Node object at 0x7f32ca43e320>]
veracity:tensor([[1.000

AttributeError: ignored

# Metrics

In [ ]:
def val_loader(val_index):
  for e in [l for i,l in enumerate(event_labels) if i==val_index]:
    with open('/content/drive/My Drive/DeepMoji/data_bert_{}.pkl'.format(e),'rb') as f:
      ev=pickle.load(f)
    yield ev,e

model=TreeLSTMCell(768,768)
model.load_state_dict(torch.load('/content/drive/My Drive/model_bert.pt'))
results={}
for val_index in range(5):
  with torch.no_grad():
    for event,lbl in val_loader(val_index):
      pred=[]
      print('testing on {}...'.format(lbl))
      for tree in event.treelist:
        nodes=tree[1]
        for i in range(len(nodes)):
          nodes[i].h=None
          nodes[i].c=None
        model.reset_h_c()  
        for i in range(len(nodes)):
          # if(nodes[i].x is not None):
            # print(nodes[i].x.tolist()[0][0],nodes[i].id)
          if(i!=(len(nodes)-1)):
            result=model(nodes[i])
            
          else:
            result=model(nodes[i],is_last=True)
            # print(result)
            pred.append((torch.argmax(result).item(),veracity_dict[str(tree[0])]))
      results[lbl]=pred        

testing on charliehebdo...


NameError: ignored

In [ ]:
ev_label_count=[]
for e in ['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']:
  with open('/content/drive/My Drive/DeepMoji/data_bert_{}.pkl'.format(e),'rb') as f:
    ev=pickle.load(f)
  label={0:0,1:0,2:0,3:0}   
  # for event in ev:
  for tree in ev.treelist:
    nodes=tree[1]
    for node in nodes:
      if(node.y is not None):
        label[node.y]+=1
  ev_label_count.append((e,label))    
print(ev_label_count)

[('charliehebdo', {0: 169, 1: 719, 2: 53, 3: 56}), ('sydneysiege', {0: 150, 1: 700, 2: 98, 3: 88}), ('germanwings-crash', {0: 46, 1: 172, 2: 28, 3: 10}), ('ferguson', {0: 138, 1: 713, 2: 99, 3: 90}), ('ottawashooting', {0: 105, 1: 477, 2: 63, 3: 74})]
